In [1]:
#Establish required libraries

import sys
import types
import pandas as pd 
from botocore.client import Config
import ibm_boto3


import numpy as np
import pandas as pd
from pandas import Series, DataFrame, Index
from IPython.display import Image

def __iter__(self): return 0

import pandas as pd
import numpy as np 

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)


In [2]:
# The code was removed by Watson Studio for sharing.

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385636 entries, 0 to 385635
Data columns (total 72 columns):
Demo_Key                                    385636 non-null int64
Promo_ID                                    385636 non-null object
Promo_CPG_Name                              281228 non-null object
Promo_Name                                  385636 non-null object
Retailer_Promo_Contract_ID                  0 non-null float64
Retailer_Name                               385636 non-null object
Retailer_Store_ID                           385636 non-null object
Product_Unit_UPC                            385636 non-null object
Product_Group_ID                            385635 non-null object
Product_Brand                               385636 non-null object
Product_Brand_Category_Desc                 385636 non-null object
Product_Category                            385636 non-null object
Product_Sub_Category                        385636 non-null object
Product_Type           

In [3]:
#select the fields you need
df_price_elasticity = df_data_1[['Promo_ID','Promo_Name','Retailer_Name','Retailer_Store_ID','Product_Category',
                                 'Product_Sub_Category',
                             'Product_Unit_UPC','Product_Group_ID','Product_Brand','Product_Brand_Category_Desc',
                              'Product_Unit_Description','Retailer_Product_Base_Day_Unit_Volume',
                              'TPP_Unit_Sale_Price','Retailer_Product_Regular_Unit_Price','TPP_Incremental_Cases',
                             'TPP_Baseline_Cases_Volume','TPP_Type','TPP_Start','TPP_End','Product_Units_Per_Case','Product_CPG_COGS_Price']]






In [4]:
#Visually inspect the different products in the table
df_price_elasticity.Product_Sub_Category.value_counts(dropna = False).sort_index()

Cereal         67360
Cleaners       30312
Ice Cream      47152
Trail Mix      74096
Vegetables    116196
Yogurt         50520
Name: Product_Sub_Category, dtype: int64

In [5]:
df_price_elasticity.shape

(385636, 21)

In [6]:
#De-Dup the file
df_price_elasticity=df_price_elasticity.drop_duplicates(subset=['Promo_ID','Retailer_Store_ID','Product_Unit_UPC'], keep='first', inplace=False)
df_price_elasticity.shape




(385636, 21)

In [7]:
#remove all products but the one we us

df_price_elasticity=df_price_elasticity[df_price_elasticity['Product_Sub_Category'] =='Cereal']
df_price_elasticity.shape

(67360, 21)

In [8]:
#create a new field for size
df_price_elasticity['SIZE']=pd.DataFrame(df_price_elasticity['Product_Group_ID'].str[-2:])


In [9]:
df_price_elasticity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67360 entries, 0 to 67359
Data columns (total 22 columns):
Promo_ID                                 67360 non-null object
Promo_Name                               67360 non-null object
Retailer_Name                            67360 non-null object
Retailer_Store_ID                        67360 non-null object
Product_Category                         67360 non-null object
Product_Sub_Category                     67360 non-null object
Product_Unit_UPC                         67360 non-null object
Product_Group_ID                         67360 non-null object
Product_Brand                            67360 non-null object
Product_Brand_Category_Desc              67360 non-null object
Product_Unit_Description                 67360 non-null object
Retailer_Product_Base_Day_Unit_Volume    67360 non-null float64
TPP_Unit_Sale_Price                      67360 non-null float64
Retailer_Product_Regular_Unit_Price      67360 non-null float64
TPP_In

In [10]:
#create a copy of the original data set
sale=df_price_elasticity

In [11]:
sale.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67360 entries, 0 to 67359
Data columns (total 22 columns):
Promo_ID                                 67360 non-null object
Promo_Name                               67360 non-null object
Retailer_Name                            67360 non-null object
Retailer_Store_ID                        67360 non-null object
Product_Category                         67360 non-null object
Product_Sub_Category                     67360 non-null object
Product_Unit_UPC                         67360 non-null object
Product_Group_ID                         67360 non-null object
Product_Brand                            67360 non-null object
Product_Brand_Category_Desc              67360 non-null object
Product_Unit_Description                 67360 non-null object
Retailer_Product_Base_Day_Unit_Volume    67360 non-null float64
TPP_Unit_Sale_Price                      67360 non-null float64
Retailer_Product_Regular_Unit_Price      67360 non-null float64
TPP_In

In [12]:
# create rename the sale price to price, create the cases and units field, drop fields you don't need anymore
#label all of the records as sale records

sale=sale.rename(index=str, columns={"TPP_Unit_Sale_Price": "PRICE"})
sale['CASES'] = sale['TPP_Baseline_Cases_Volume'] + sale['TPP_Incremental_Cases'] 
sale['GROUP']='ONSALE'
sale['UNITS']=sale['CASES']*sale['Product_Units_Per_Case']
sale=sale.drop(['TPP_Baseline_Cases_Volume', 'TPP_Incremental_Cases','Retailer_Product_Regular_Unit_Price','Retailer_Product_Base_Day_Unit_Volume'], axis=1)

In [13]:
sale.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67360 entries, 0 to 67359
Data columns (total 21 columns):
Promo_ID                       67360 non-null object
Promo_Name                     67360 non-null object
Retailer_Name                  67360 non-null object
Retailer_Store_ID              67360 non-null object
Product_Category               67360 non-null object
Product_Sub_Category           67360 non-null object
Product_Unit_UPC               67360 non-null object
Product_Group_ID               67360 non-null object
Product_Brand                  67360 non-null object
Product_Brand_Category_Desc    67360 non-null object
Product_Unit_Description       67360 non-null object
PRICE                          67360 non-null float64
TPP_Type                       67360 non-null object
TPP_Start                      67360 non-null object
TPP_End                        67360 non-null object
Product_Units_Per_Case         67360 non-null float64
Product_CPG_COGS_Price         67360 non-null 

In [14]:
#copy the orginal data set, create the non-sale prices
nosale=df_price_elasticity


In [15]:
nosale.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67360 entries, 0 to 67359
Data columns (total 22 columns):
Promo_ID                                 67360 non-null object
Promo_Name                               67360 non-null object
Retailer_Name                            67360 non-null object
Retailer_Store_ID                        67360 non-null object
Product_Category                         67360 non-null object
Product_Sub_Category                     67360 non-null object
Product_Unit_UPC                         67360 non-null object
Product_Group_ID                         67360 non-null object
Product_Brand                            67360 non-null object
Product_Brand_Category_Desc              67360 non-null object
Product_Unit_Description                 67360 non-null object
Retailer_Product_Base_Day_Unit_Volume    67360 non-null float64
TPP_Unit_Sale_Price                      67360 non-null float64
Retailer_Product_Regular_Unit_Price      67360 non-null float64
TPP_In

In [16]:
#create the cases and price field
nosale=nosale.rename(index=str, columns={"TPP_Baseline_Cases_Volume": "CASES", "Retailer_Product_Regular_Unit_Price":"PRICE"})


In [17]:
nosale.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67360 entries, 0 to 67359
Data columns (total 22 columns):
Promo_ID                                 67360 non-null object
Promo_Name                               67360 non-null object
Retailer_Name                            67360 non-null object
Retailer_Store_ID                        67360 non-null object
Product_Category                         67360 non-null object
Product_Sub_Category                     67360 non-null object
Product_Unit_UPC                         67360 non-null object
Product_Group_ID                         67360 non-null object
Product_Brand                            67360 non-null object
Product_Brand_Category_Desc              67360 non-null object
Product_Unit_Description                 67360 non-null object
Retailer_Product_Base_Day_Unit_Volume    67360 non-null float64
TPP_Unit_Sale_Price                      67360 non-null float64
PRICE                                    67360 non-null float64
TPP_Increme

In [18]:
#Calculate the number of Units
nosale['UNITS']=nosale['CASES']*nosale['Product_Units_Per_Case']

In [19]:
#identify the prices as non-sale prices.
nosale['GROUP']='NOTONSALE'


#Remove stuff you don't need


nosale=nosale.drop([ 'TPP_Incremental_Cases','TPP_Unit_Sale_Price','Retailer_Product_Base_Day_Unit_Volume','TPP_Type'], axis=1)
nosale['TPP_Type']='N'

In [20]:
nosale.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Category,Product_Sub_Category,Product_Unit_UPC,Product_Group_ID,Product_Brand,Product_Brand_Category_Desc,...,PRICE,CASES,TPP_Start,TPP_End,Product_Units_Per_Case,Product_CPG_COGS_Price,SIZE,UNITS,GROUP,TPP_Type
0,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,Breakfast & Cereal,Cereal,55444-60215,BB-BC-C-CC-15,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,...,4.21,2.0,2019-06-05,2019-06-13,18.0,3.06,15,36.0,NOTONSALE,N
1,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,Breakfast & Cereal,Cereal,55444-60315,BB-BC-C-CC-15,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,...,4.21,2.0,2019-06-05,2019-06-13,18.0,3.06,15,36.0,NOTONSALE,N
2,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,Breakfast & Cereal,Cereal,55444-60415,BB-BC-C-CC-15,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,...,4.21,2.0,2019-06-05,2019-06-13,18.0,3.06,15,36.0,NOTONSALE,N
3,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,Breakfast & Cereal,Cereal,55444-60515,BB-BC-C-CC-15,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,...,4.21,2.0,2019-06-05,2019-06-13,18.0,3.06,15,36.0,NOTONSALE,N
4,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,Breakfast & Cereal,Cereal,55444-60615,BB-BC-C-CC-15,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,...,4.21,2.0,2019-06-05,2019-06-13,18.0,3.06,15,36.0,NOTONSALE,N


In [21]:
sale.info()
nosale.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67360 entries, 0 to 67359
Data columns (total 21 columns):
Promo_ID                       67360 non-null object
Promo_Name                     67360 non-null object
Retailer_Name                  67360 non-null object
Retailer_Store_ID              67360 non-null object
Product_Category               67360 non-null object
Product_Sub_Category           67360 non-null object
Product_Unit_UPC               67360 non-null object
Product_Group_ID               67360 non-null object
Product_Brand                  67360 non-null object
Product_Brand_Category_Desc    67360 non-null object
Product_Unit_Description       67360 non-null object
PRICE                          67360 non-null float64
TPP_Type                       67360 non-null object
TPP_Start                      67360 non-null object
TPP_End                        67360 non-null object
Product_Units_Per_Case         67360 non-null float64
Product_CPG_COGS_Price         67360 non-null 

In [22]:
#include needed fields
nosale = nosale[['Promo_ID',
 'Promo_Name',
 'Retailer_Name',
 'Retailer_Store_ID',
 'Product_Group_ID',
 'Product_Unit_UPC',
 'Product_Brand',
 'Product_Brand_Category_Desc',
 'Product_Unit_Description',
 'PRICE',
 'TPP_Type',
 'TPP_Start',
 'TPP_End',
 'Product_Units_Per_Case',
 'Product_CPG_COGS_Price',
 'SIZE',
 'CASES',
 'GROUP',
 'UNITS']]

In [23]:
#include needed fields
sale = sale[['Promo_ID',
 'Promo_Name',
 'Retailer_Name',
 'Retailer_Store_ID',
 'Product_Group_ID',
 'Product_Unit_UPC',
 'Product_Brand',
 'Product_Brand_Category_Desc',
 'Product_Unit_Description',
 'PRICE',
 'TPP_Type',
 'TPP_Start',
 'TPP_End',
 'Product_Units_Per_Case',
 'Product_CPG_COGS_Price',
 'SIZE',
 'CASES',
 'GROUP',
 'UNITS']]

In [24]:
sale.info()
nosale.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67360 entries, 0 to 67359
Data columns (total 19 columns):
Promo_ID                       67360 non-null object
Promo_Name                     67360 non-null object
Retailer_Name                  67360 non-null object
Retailer_Store_ID              67360 non-null object
Product_Group_ID               67360 non-null object
Product_Unit_UPC               67360 non-null object
Product_Brand                  67360 non-null object
Product_Brand_Category_Desc    67360 non-null object
Product_Unit_Description       67360 non-null object
PRICE                          67360 non-null float64
TPP_Type                       67360 non-null object
TPP_Start                      67360 non-null object
TPP_End                        67360 non-null object
Product_Units_Per_Case         67360 non-null float64
Product_CPG_COGS_Price         67360 non-null float64
SIZE                           67360 non-null object
CASES                          67360 non-null

In [25]:
#append the sale and non-sale records into one dataframe
df_total=pd.DataFrame(sale.append(nosale, ignore_index=True, verify_integrity=False))

In [26]:
#convert the tpp_start which is character to a pandas date field
df_total['date'] = pd.to_datetime(df_total['TPP_Start'])
df_total.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,TPP_Type,TPP_Start,TPP_End,Product_Units_Per_Case,Product_CPG_COGS_Price,SIZE,CASES,GROUP,UNITS,date
0,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,F,2019-06-05,2019-06-13,18.0,3.06,15,3.0,ONSALE,54.0,2019-06-05
1,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60315,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Vanilla Light and Crunchy 15 oz,3.57,F,2019-06-05,2019-06-13,18.0,3.06,15,3.0,ONSALE,54.0,2019-06-05
2,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60415,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Honey Gone Nuts 15 oz,3.57,F,2019-06-05,2019-06-13,18.0,3.06,15,3.0,ONSALE,54.0,2019-06-05
3,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60515,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Cherry Almond 15 oz,3.57,F,2019-06-05,2019-06-13,18.0,3.06,15,3.0,ONSALE,54.0,2019-06-05
4,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60615,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Organic Maple 15 oz,3.57,F,2019-06-05,2019-06-13,18.0,3.06,15,3.0,ONSALE,54.0,2019-06-05


In [27]:
#create a field for year and month
df_total['year'], df_total['month'] = df_total['date'].dt.year, df_total['date'].dt.month
df_total.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,TPP_End,Product_Units_Per_Case,Product_CPG_COGS_Price,SIZE,CASES,GROUP,UNITS,date,year,month
0,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,2019-06-13,18.0,3.06,15,3.0,ONSALE,54.0,2019-06-05,2019,6
1,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60315,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Vanilla Light and Crunchy 15 oz,3.57,...,2019-06-13,18.0,3.06,15,3.0,ONSALE,54.0,2019-06-05,2019,6
2,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60415,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Honey Gone Nuts 15 oz,3.57,...,2019-06-13,18.0,3.06,15,3.0,ONSALE,54.0,2019-06-05,2019,6
3,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60515,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Cherry Almond 15 oz,3.57,...,2019-06-13,18.0,3.06,15,3.0,ONSALE,54.0,2019-06-05,2019,6
4,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60615,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Organic Maple 15 oz,3.57,...,2019-06-13,18.0,3.06,15,3.0,ONSALE,54.0,2019-06-05,2019,6


In [28]:
#assign random numbers to each retailer
wookie = pd.DataFrame(df_total.groupby(['Retailer_Store_ID'])['PRICE'].mean())
wookie=wookie.drop(['PRICE'], axis=1)


In [29]:
wookie['Retailer_R'] = (np.random.randint(-30, 60, wookie.shape[0]))/1000
wookie.reset_index(level=0, inplace=True)
wookie.head()


,Retailer_Store_ID,Retailer_R
0,12345-10024-126,-0.017
1,12345-14213-136,0.037
2,12345-14601-157,0.037
3,12345-15225-129,-0.023
4,12345-15225-167,0.054


In [30]:
df_total1 =df_total.merge(wookie, on=['Retailer_Store_ID'], how='inner')
df_total1.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,Product_Units_Per_Case,Product_CPG_COGS_Price,SIZE,CASES,GROUP,UNITS,date,year,month,Retailer_R
0,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,18.0,3.06,15,3.0,ONSALE,54.0,2019-06-05,2019,6,0.037
1,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60315,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Vanilla Light and Crunchy 15 oz,3.57,...,18.0,3.06,15,3.0,ONSALE,54.0,2019-06-05,2019,6,0.037
2,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60415,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Honey Gone Nuts 15 oz,3.57,...,18.0,3.06,15,3.0,ONSALE,54.0,2019-06-05,2019,6,0.037
3,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60515,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Cherry Almond 15 oz,3.57,...,18.0,3.06,15,3.0,ONSALE,54.0,2019-06-05,2019,6,0.037
4,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60615,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Organic Maple 15 oz,3.57,...,18.0,3.06,15,3.0,ONSALE,54.0,2019-06-05,2019,6,0.037


In [31]:
#assign random numbers to each product group id
df_total1.Product_Group_ID.value_counts(dropna = False).sort_index()

BB-BC-C-CC-15    67360
BB-BC-C-CC-24    67360
Name: Product_Group_ID, dtype: int64

In [32]:
 wookie = pd.DataFrame(df_total1.groupby(['Product_Group_ID'])['PRICE'].mean())
wookie=wookie.drop(['PRICE'], axis=1)

In [33]:
wookie['PRODUCT_GROUP_ID_R'] = (np.random.randint(-30, 60, wookie.shape[0]))/1000
wookie.reset_index(level=0, inplace=True)
wookie.head()

,Product_Group_ID,PRODUCT_GROUP_ID_R
0,BB-BC-C-CC-15,-0.027
1,BB-BC-C-CC-24,0.050


In [34]:
#!pip install plotly --upgrade

In [35]:
df_total2 =df_total1.merge(wookie, on=['Product_Group_ID'], how='inner')
df_total2.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,Product_CPG_COGS_Price,SIZE,CASES,GROUP,UNITS,date,year,month,Retailer_R,PRODUCT_GROUP_ID_R
0,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,3.06,15,3.0,ONSALE,54.0,2019-06-05,2019,6,0.037,-0.027
1,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60315,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Vanilla Light and Crunchy 15 oz,3.57,...,3.06,15,3.0,ONSALE,54.0,2019-06-05,2019,6,0.037,-0.027
2,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60415,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Honey Gone Nuts 15 oz,3.57,...,3.06,15,3.0,ONSALE,54.0,2019-06-05,2019,6,0.037,-0.027
3,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60515,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Cherry Almond 15 oz,3.57,...,3.06,15,3.0,ONSALE,54.0,2019-06-05,2019,6,0.037,-0.027
4,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60615,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Organic Maple 15 oz,3.57,...,3.06,15,3.0,ONSALE,54.0,2019-06-05,2019,6,0.037,-0.027


In [36]:
#assign random numbers to each product unit upc value
df_total2.Product_Unit_UPC.value_counts(dropna = False).sort_index()

55444-60215    3368
55444-60315    3368
55444-60415    3368
55444-60515    3368
55444-60615    3368
55444-60715    3368
55444-60815    3368
55444-60915    3368
55444-61015    3368
55444-61115    3368
55444-61215    3368
55444-61315    3368
55444-61415    3368
55444-61515    3368
55444-61615    3368
55444-61715    3368
55444-61815    3368
55444-61915    3368
55444-62015    3368
55444-62115    3368
55444-62224    3368
55444-62324    3368
55444-62424    3368
55444-62524    3368
55444-62624    3368
55444-62724    3368
55444-62824    3368
55444-62924    3368
55444-63024    3368
55444-63124    3368
55444-63224    3368
55444-63324    3368
55444-63424    3368
55444-63524    3368
55444-63624    3368
55444-63724    3368
55444-63824    3368
55444-63924    3368
55444-64024    3368
55444-64124    3368
Name: Product_Unit_UPC, dtype: int64

In [37]:
wookie = pd.DataFrame(df_total2.groupby(['Product_Unit_UPC'])['PRICE'].mean())
wookie=wookie.drop(['PRICE'], axis=1)

wookie['Product_Unit_UPC_R'] = (np.random.randint(-30, 60, wookie.shape[0]))/1000
wookie.reset_index(level=0, inplace=True)
wookie.head()
df_total3 =df_total2.merge(wookie, on=['Product_Unit_UPC'], how='inner')
df_total3.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,SIZE,CASES,GROUP,UNITS,date,year,month,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R
0,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,15,3.0,ONSALE,54.0,2019-06-05,2019,6,0.037,-0.027,0.025
1,Brandega-43771,Early Risers,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,15,2.0,ONSALE,36.0,2019-09-09,2019,9,0.037,-0.027,0.025
2,Brandega-68683,Giving Thanks,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,15,4.0,ONSALE,72.0,2019-11-07,2019,11,0.037,-0.027,0.025
3,Brandega-74381,New Year New You,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,15,4.0,ONSALE,72.0,2019-01-12,2019,1,0.037,-0.027,0.025
4,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,4.21,...,15,2.0,NOTONSALE,36.0,2019-06-05,2019,6,0.037,-0.027,0.025


In [38]:
#assign random numbers to each promo
df_total2.Promo_Name.value_counts(dropna = False).sort_index()

Early Risers        33680
Giving Thanks       33680
Helpful Starters    33680
New Year New You    33680
Name: Promo_Name, dtype: int64

In [39]:
wookie = pd.DataFrame(df_total3.groupby(['Promo_Name'])['PRICE'].mean())
wookie=wookie.drop(['PRICE'], axis=1)

wookie['Promo_Name_R'] = (np.random.randint(-30, 60, wookie.shape[0]))/1000
wookie.reset_index(level=0, inplace=True)
wookie.head()
df_total4 =df_total3.merge(wookie, on=['Promo_Name'], how='inner')
df_total4.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,CASES,GROUP,UNITS,date,year,month,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R
0,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,3.0,ONSALE,54.0,2019-06-05,2019,6,0.037,-0.027,0.025,-0.003
1,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,4.21,...,2.0,NOTONSALE,36.0,2019-06-05,2019,6,0.037,-0.027,0.025,-0.003
2,Brandega-24816,Helpful Starters,Botann's,12345-79914-108,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,3.0,ONSALE,54.0,2019-06-05,2019,6,0.057,-0.027,0.025,-0.003
3,Brandega-24816,Helpful Starters,Botann's,12345-79914-108,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,4.21,...,2.0,NOTONSALE,36.0,2019-06-05,2019,6,0.057,-0.027,0.025,-0.003
4,Brandega-24816,Helpful Starters,Botann's,12345-90005-109,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,3.0,ONSALE,54.0,2019-06-05,2019,6,0.054,-0.027,0.025,-0.003


In [40]:
#assign different values to sale and non-sale records
df_total4['GROUP_R'] = np.where(df_total4['GROUP'] == 'ONSALE', -0.10, 0.01)
df_total4.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,GROUP,UNITS,date,year,month,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R,GROUP_R
0,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,ONSALE,54.0,2019-06-05,2019,6,0.037,-0.027,0.025,-0.003,-0.10
1,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,4.21,...,NOTONSALE,36.0,2019-06-05,2019,6,0.037,-0.027,0.025,-0.003,0.01
2,Brandega-24816,Helpful Starters,Botann's,12345-79914-108,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,ONSALE,54.0,2019-06-05,2019,6,0.057,-0.027,0.025,-0.003,-0.10
3,Brandega-24816,Helpful Starters,Botann's,12345-79914-108,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,4.21,...,NOTONSALE,36.0,2019-06-05,2019,6,0.057,-0.027,0.025,-0.003,0.01
4,Brandega-24816,Helpful Starters,Botann's,12345-90005-109,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,ONSALE,54.0,2019-06-05,2019,6,0.054,-0.027,0.025,-0.003,-0.10


In [41]:
df_total4.month.value_counts(dropna = False).sort_index()

1     33680
6     33680
9     33680
11    33680
Name: month, dtype: int64

In [42]:
#assign values to each month
conditions = [
    (df_total4['month'] == 2),
    (df_total4['month'] ==4),
    (df_total4['month'] ==7)]
choices = [-0.03, 0.02, 0.06]
df_total4['month_R'] = np.select(conditions, choices, default=0.02)


In [43]:
df_total4.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,UNITS,date,year,month,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R,GROUP_R,month_R
0,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,54.0,2019-06-05,2019,6,0.037,-0.027,0.025,-0.003,-0.10,0.02
1,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,4.21,...,36.0,2019-06-05,2019,6,0.037,-0.027,0.025,-0.003,0.01,0.02
2,Brandega-24816,Helpful Starters,Botann's,12345-79914-108,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,54.0,2019-06-05,2019,6,0.057,-0.027,0.025,-0.003,-0.10,0.02
3,Brandega-24816,Helpful Starters,Botann's,12345-79914-108,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,4.21,...,36.0,2019-06-05,2019,6,0.057,-0.027,0.025,-0.003,0.01,0.02
4,Brandega-24816,Helpful Starters,Botann's,12345-90005-109,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,54.0,2019-06-05,2019,6,0.054,-0.027,0.025,-0.003,-0.10,0.02


In [44]:
#generate a random number for each row that is either 1 or -1
import random
df_total4['randNumCol'] = np.random.choice([1, -1], df_total4.shape[0])
df_total4.head(100)

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,date,year,month,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R,GROUP_R,month_R,randNumCol
0,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,2019-06-05,2019,6,0.037,-0.027,0.025,-0.003,-0.10,0.02,1
1,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,4.21,...,2019-06-05,2019,6,0.037,-0.027,0.025,-0.003,0.01,0.02,1
2,Brandega-24816,Helpful Starters,Botann's,12345-79914-108,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,2019-06-05,2019,6,0.057,-0.027,0.025,-0.003,-0.10,0.02,-1
3,Brandega-24816,Helpful Starters,Botann's,12345-79914-108,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,4.21,...,2019-06-05,2019,6,0.057,-0.027,0.025,-0.003,0.01,0.02,-1
4,Brandega-24816,Helpful Starters,Botann's,12345-90005-109,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,2019-06-05,2019,6,0.054,-0.027,0.025,-0.003,-0.10,0.02,1
5,Brandega-24816,Helpful Starters,Botann's,12345-90005-109,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,4.21,...,2019-06-05,2019,6,0.054,-0.027,0.025,-0.003,0.01,0.02,1
6,Brandega-24816,Helpful Starters,Botann's,12345-33233-110,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,2019-06-05,2019,6,0.050,-0.027,0.025,-0.003,-0.10,0.02,1
7,Brandega-24816,Helpful Starters,Botann's,12345-33233-110,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,4.21,...,2019-06-05,2019,6,0.050,-0.027,0.025,-0.003,0.01,0.02,-1
8,Brandega-24816,Helpful Starters,Botann's,12345-28816-111,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,2019-06-05,2019,6,-0.009,-0.027,0.025,-0.003,-0.10,0.02,-1
9,Brandega-24816,Helpful Starters,Botann's,12345-28816-111,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,4.21,...,2019-06-05,2019,6,-0.009,-0.027,0.025,-0.003,0.01,0.02,-1


In [45]:
#sum the generated numbers for each row, making a unique value for each row called total_r

cols = ['Retailer_R','PRODUCT_GROUP_ID_R','Product_Unit_UPC_R','Promo_Name_R','GROUP_R','month_R']
df_total4['TOTAL_R'] = df_total4[cols].astype(float).sum(axis=1)


In [46]:
df_total4.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,year,month,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R,GROUP_R,month_R,randNumCol,TOTAL_R
0,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,2019,6,0.037,-0.027,0.025,-0.003,-0.10,0.02,1,-0.048
1,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,4.21,...,2019,6,0.037,-0.027,0.025,-0.003,0.01,0.02,1,0.062
2,Brandega-24816,Helpful Starters,Botann's,12345-79914-108,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,2019,6,0.057,-0.027,0.025,-0.003,-0.10,0.02,-1,-0.028
3,Brandega-24816,Helpful Starters,Botann's,12345-79914-108,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,4.21,...,2019,6,0.057,-0.027,0.025,-0.003,0.01,0.02,-1,0.082
4,Brandega-24816,Helpful Starters,Botann's,12345-90005-109,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,2019,6,0.054,-0.027,0.025,-0.003,-0.10,0.02,1,-0.031


In [47]:
df_total4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134720 entries, 0 to 134719
Data columns (total 30 columns):
Promo_ID                       134720 non-null object
Promo_Name                     134720 non-null object
Retailer_Name                  134720 non-null object
Retailer_Store_ID              134720 non-null object
Product_Group_ID               134720 non-null object
Product_Unit_UPC               134720 non-null object
Product_Brand                  134720 non-null object
Product_Brand_Category_Desc    134720 non-null object
Product_Unit_Description       134720 non-null object
PRICE                          134720 non-null float64
TPP_Type                       134720 non-null object
TPP_Start                      134720 non-null object
TPP_End                        134720 non-null object
Product_Units_Per_Case         134720 non-null float64
Product_CPG_COGS_Price         134720 non-null float64
SIZE                           134720 non-null object
CASES                 

In [48]:
#create new prices based on the adjustment numbers generated on the previous lines
df_total4['PRICE_R']=df_total4['PRICE']*(1.0-(df_total4['randNumCol']*df_total4['TOTAL_R']))
df_total4['UNITS_R']=df_total4['UNITS']*(1.0+(df_total4['randNumCol']*df_total4['TOTAL_R']))

In [49]:
df_total4.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R,GROUP_R,month_R,randNumCol,TOTAL_R,PRICE_R,UNITS_R
0,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,0.037,-0.027,0.025,-0.003,-0.10,0.02,1,-0.048,3.74136,51.408
1,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,4.21,...,0.037,-0.027,0.025,-0.003,0.01,0.02,1,0.062,3.94898,38.232
2,Brandega-24816,Helpful Starters,Botann's,12345-79914-108,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,0.057,-0.027,0.025,-0.003,-0.10,0.02,-1,-0.028,3.47004,55.512
3,Brandega-24816,Helpful Starters,Botann's,12345-79914-108,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,4.21,...,0.057,-0.027,0.025,-0.003,0.01,0.02,-1,0.082,4.55522,33.048
4,Brandega-24816,Helpful Starters,Botann's,12345-90005-109,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,0.054,-0.027,0.025,-0.003,-0.10,0.02,1,-0.031,3.68067,52.326


In [50]:
#Make sure that size is always positive, create a new variable called price_size convert size to a numeric value
df_total4.SIZE = df_total4.SIZE.astype(float).fillna(0.0)


df_total4['PRICE_SIZE']=df_total4['PRICE_R']*df_total4['SIZE']


df_total4.SIZE.value_counts(dropna = False).sort_index()

15.0    67360
24.0    67360
Name: SIZE, dtype: int64

In [51]:
#create a dummy variable for ice cream
df_total4['SIZE_16'] = np.where(df_total4['SIZE'] == 16, 1, 0)
df_total4['SIZE_16_PRICE']=df_total4['SIZE_16']*df_total4['PRICE']
df_total4.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,Promo_Name_R,GROUP_R,month_R,randNumCol,TOTAL_R,PRICE_R,UNITS_R,PRICE_SIZE,SIZE_16,SIZE_16_PRICE
0,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,-0.003,-0.10,0.02,1,-0.048,3.74136,51.408,56.12040,0,0.0
1,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,4.21,...,-0.003,0.01,0.02,1,0.062,3.94898,38.232,59.23470,0,0.0
2,Brandega-24816,Helpful Starters,Botann's,12345-79914-108,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,-0.003,-0.10,0.02,-1,-0.028,3.47004,55.512,52.05060,0,0.0
3,Brandega-24816,Helpful Starters,Botann's,12345-79914-108,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,4.21,...,-0.003,0.01,0.02,-1,0.082,4.55522,33.048,68.32830,0,0.0
4,Brandega-24816,Helpful Starters,Botann's,12345-90005-109,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,3.57,...,-0.003,-0.10,0.02,1,-0.031,3.68067,52.326,55.21005,0,0.0


In [52]:
#bring in the stats library to build the model
import statsmodels.api as sm  
#scikit learn is another library that you could use.  Its focus is more on prediction.  This library is more focused on understanding why and how certain variables impact your target.  Which is quantity in this case


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [53]:
df_total4=df_total4.drop(['PRICE'], axis=1)

In [54]:
v = df_total4[['PRICE_R','TOTAL_R','SIZE','PRICE_SIZE','UNITS_R']]
w = df_total4[['PRICE_R','TOTAL_R','SIZE','PRICE_SIZE','UNITS_R']]
x = df_total4[['PRICE_R','TOTAL_R','SIZE','PRICE_SIZE','UNITS_R']]
y = df_total4[['PRICE_R','TOTAL_R','SIZE','PRICE_SIZE','UNITS_R']]
z = df_total4[['PRICE_R','TOTAL_R','SIZE','PRICE_SIZE','UNITS_R']]

In [55]:
pricev = pd.DataFrame((v['PRICE_R']*0.95))
pricew = pd.DataFrame((w['PRICE_R']*0.85))
pricey = pd.DataFrame((y['PRICE_R']*0.90))
pricez= pd.DataFrame((z['PRICE_R']*0.80))

qv = pd.DataFrame((v['UNITS_R']*1.05))
qw = pd.DataFrame((w['UNITS_R']*1.15))
qy = pd.DataFrame((y['UNITS_R']*1.10))
qz= pd.DataFrame((z['UNITS_R']*1.20))

In [56]:
df_total4.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,TPP_Type,...,Promo_Name_R,GROUP_R,month_R,randNumCol,TOTAL_R,PRICE_R,UNITS_R,PRICE_SIZE,SIZE_16,SIZE_16_PRICE
0,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,F,...,-0.003,-0.10,0.02,1,-0.048,3.74136,51.408,56.12040,0,0.0
1,Brandega-24816,Helpful Starters,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,N,...,-0.003,0.01,0.02,1,0.062,3.94898,38.232,59.23470,0,0.0
2,Brandega-24816,Helpful Starters,Botann's,12345-79914-108,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,F,...,-0.003,-0.10,0.02,-1,-0.028,3.47004,55.512,52.05060,0,0.0
3,Brandega-24816,Helpful Starters,Botann's,12345-79914-108,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,N,...,-0.003,0.01,0.02,-1,0.082,4.55522,33.048,68.32830,0,0.0
4,Brandega-24816,Helpful Starters,Botann's,12345-90005-109,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,F,...,-0.003,-0.10,0.02,1,-0.031,3.68067,52.326,55.21005,0,0.0


In [57]:
v=v.drop(['PRICE_R','UNITS_R'], axis=1)
v = pd.concat([v, pricev,qv], axis=1)
v['PRICE_POINT']='V'
stuff=df_total4[['Promo_ID','Retailer_Store_ID','Product_Unit_UPC','TPP_Type']]
v = pd.concat([v, stuff], axis=1)

In [58]:

w=w.drop(['PRICE_R','UNITS_R'], axis=1)
w = pd.concat([w, pricew,qw], axis=1)
w['PRICE_POINT']='W'

w = pd.concat([w, stuff], axis=1)

y=y.drop(['PRICE_R','UNITS_R'], axis=1)
y = pd.concat([y, pricey,qy], axis=1)
y['PRICE_POINT']='Y'
y = pd.concat([y, stuff], axis=1)

z=z.drop(['PRICE_R','UNITS_R'], axis=1)
z = pd.concat([z, pricez,qz], axis=1)
z['PRICE_POINT']='Z'
z = pd.concat([z, stuff], axis=1)

x['PRICE_POINT']='X'
x = pd.concat([x, stuff], axis=1)





/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [59]:
xxx=pd.concat([v,w,x,y,z],axis=0)

In [60]:
#build the ols model

independent=xxx[['PRICE_R','TOTAL_R','SIZE','PRICE_SIZE']]
P=xxx['PRICE_POINT']
stuff=xxx[['Promo_ID','Retailer_Store_ID','Product_Unit_UPC','TPP_Type']]

independent = sm.add_constant(independent, prepend=False)

dependent=xxx['UNITS_R']

In [61]:
mod = sm.OLS(dependent, independent)

In [62]:
res = mod.fit()

In [63]:
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                UNITS_R   R-squared:                       0.544
Model:                            OLS   Adj. R-squared:                  0.544
Method:                 Least Squares   F-statistic:                 2.005e+05
Date:                Thu, 17 Jan 2019   Prob (F-statistic):               0.00
Time:                        17:38:18   Log-Likelihood:            -2.8279e+06
No. Observations:              673600   AIC:                         5.656e+06
Df Residuals:                  673595   BIC:                         5.656e+06
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
PRICE_R       -8.1717      0.047   -173.650      0.0

In [64]:
ypred = pd.DataFrame(res.predict(independent))
ypred.columns = ['P_UNITS']

In [65]:
model_output=pd.concat([ypred,independent,stuff,P], axis=1)
model_output.head()

,P_UNITS,PRICE_R,TOTAL_R,SIZE,PRICE_SIZE,const,Promo_ID,Retailer_Store_ID,Product_Unit_UPC,TPP_Type,PRICE_POINT
0,55.473526,3.554292,-0.048,15.0,56.12040,1.0,Brandega-24816,12345-94249-107,55444-60215,F,V
1,34.243312,3.751531,0.062,15.0,59.23470,1.0,Brandega-24816,12345-94249-107,55444-60215,N,V
2,54.796160,3.296538,-0.028,15.0,52.05060,1.0,Brandega-24816,12345-79914-108,55444-60215,F,V
3,24.529175,4.327459,0.082,15.0,68.32830,1.0,Brandega-24816,12345-79914-108,55444-60215,N,V
4,53.147871,3.496636,-0.031,15.0,55.21005,1.0,Brandega-24816,12345-90005-109,55444-60215,F,V


In [66]:
df_main_stuff=df_total4.drop(['PRICE_R','TOTAL_R','SIZE','PRICE_SIZE'], axis=1)
df_main_stuff.columns

Index(['Promo_ID', 'Promo_Name', 'Retailer_Name', 'Retailer_Store_ID',
       'Product_Group_ID', 'Product_Unit_UPC', 'Product_Brand',
       'Product_Brand_Category_Desc', 'Product_Unit_Description', 'TPP_Type',
       'TPP_Start', 'TPP_End', 'Product_Units_Per_Case',
       'Product_CPG_COGS_Price', 'CASES', 'GROUP', 'UNITS', 'date', 'year',
       'month', 'Retailer_R', 'PRODUCT_GROUP_ID_R', 'Product_Unit_UPC_R',
       'Promo_Name_R', 'GROUP_R', 'month_R', 'randNumCol', 'UNITS_R',
       'SIZE_16', 'SIZE_16_PRICE'],
      dtype='object')

In [67]:
df_main_stuff=df_main_stuff.drop_duplicates(subset=['Promo_ID','Retailer_Store_ID','Product_Unit_UPC','TPP_Type'], keep='first', inplace=False)
df_main_stuff.shape

(134720, 30)

In [68]:
df_total5 =model_output.merge(df_main_stuff, on=['Promo_ID','Retailer_Store_ID','Product_Unit_UPC','TPP_Type'], how='inner')
df_total5.shape

(673600, 37)

In [69]:
df_total5.describe()

,P_UNITS,PRICE_R,TOTAL_R,SIZE,PRICE_SIZE,const,Product_Units_Per_Case,Product_CPG_COGS_Price,CASES,UNITS,...,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R,GROUP_R,month_R,randNumCol,UNITS_R,SIZE_16,SIZE_16_PRICE
count,673600.000000,673600.000000,673600.000000,673600.000000,673600.000000,673600.0,673600.000000,673600.000000,673600.000000,673600.00000,...,673600.000000,673600.0000,673600.000000,673600.00000,673600.000,6.736000e+05,673600.000000,673600.000000,673600.0,673600.0
mean,45.775121,4.941332,0.034211,19.500000,114.258106,1.0,15.000000,4.320000,2.875000,41.62500,...,0.015311,0.0115,0.016650,0.01575,-0.045,2.000000e-02,-0.001960,41.613747,0.0,0.0
std,17.577271,1.629548,0.075513,4.500003,58.230391,0.0,3.000002,1.260001,1.615357,21.26288,...,0.026653,0.0385,0.021134,0.02130,0.055,2.987265e-13,0.999999,21.463850,0.0,0.0
min,-18.945601,2.361912,-0.173000,15.000000,44.285850,1.0,12.000000,3.060000,1.000000,12.00000,...,-0.030000,-0.0270,-0.030000,-0.00700,-0.100,2.000000e-02,-1.000000,9.804000,0.0,0.0
25%,33.876171,3.474723,-0.023000,15.000000,57.908550,1.0,12.000000,3.060000,2.000000,24.00000,...,-0.008000,-0.0270,0.004000,-0.00400,-0.100,2.000000e-02,-1.000000,21.288000,0.0,0.0
50%,46.610684,4.748880,0.034000,19.500000,105.232965,1.0,15.000000,4.320000,2.000000,36.00000,...,0.015000,0.0115,0.019500,0.01350,-0.045,2.000000e-02,-1.000000,35.964000,0.0,0.0
75%,60.124835,6.190684,0.091000,24.000000,161.083440,1.0,18.000000,5.580000,4.000000,63.00000,...,0.039000,0.0500,0.027750,0.03325,0.010,2.000000e-02,1.000000,63.360000,0.0,0.0
max,91.693528,9.457110,0.233000,24.000000,226.970640,1.0,18.000000,5.580000,6.000000,72.00000,...,0.059000,0.0500,0.058000,0.04300,0.010,2.000000e-02,1.000000,84.456000,0.0,0.0


In [70]:
df_total5.loc[df_total5['P_UNITS'] < 0, 'P_UNITS'] = 0
df_total5.describe()

,P_UNITS,PRICE_R,TOTAL_R,SIZE,PRICE_SIZE,const,Product_Units_Per_Case,Product_CPG_COGS_Price,CASES,UNITS,...,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R,GROUP_R,month_R,randNumCol,UNITS_R,SIZE_16,SIZE_16_PRICE
count,673600.000000,673600.000000,673600.000000,673600.000000,673600.000000,673600.0,673600.000000,673600.000000,673600.000000,673600.00000,...,673600.000000,673600.0000,673600.000000,673600.00000,673600.000,6.736000e+05,673600.000000,673600.000000,673600.0,673600.0
mean,45.812594,4.941332,0.034211,19.500000,114.258106,1.0,15.000000,4.320000,2.875000,41.62500,...,0.015311,0.0115,0.016650,0.01575,-0.045,2.000000e-02,-0.001960,41.613747,0.0,0.0
std,17.471343,1.629548,0.075513,4.500003,58.230391,0.0,3.000002,1.260001,1.615357,21.26288,...,0.026653,0.0385,0.021134,0.02130,0.055,2.987265e-13,0.999999,21.463850,0.0,0.0
min,0.000000,2.361912,-0.173000,15.000000,44.285850,1.0,12.000000,3.060000,1.000000,12.00000,...,-0.030000,-0.0270,-0.030000,-0.00700,-0.100,2.000000e-02,-1.000000,9.804000,0.0,0.0
25%,33.876171,3.474723,-0.023000,15.000000,57.908550,1.0,12.000000,3.060000,2.000000,24.00000,...,-0.008000,-0.0270,0.004000,-0.00400,-0.100,2.000000e-02,-1.000000,21.288000,0.0,0.0
50%,46.610684,4.748880,0.034000,19.500000,105.232965,1.0,15.000000,4.320000,2.000000,36.00000,...,0.015000,0.0115,0.019500,0.01350,-0.045,2.000000e-02,-1.000000,35.964000,0.0,0.0
75%,60.124835,6.190684,0.091000,24.000000,161.083440,1.0,18.000000,5.580000,4.000000,63.00000,...,0.039000,0.0500,0.027750,0.03325,0.010,2.000000e-02,1.000000,63.360000,0.0,0.0
max,91.693528,9.457110,0.233000,24.000000,226.970640,1.0,18.000000,5.580000,6.000000,72.00000,...,0.059000,0.0500,0.058000,0.04300,0.010,2.000000e-02,1.000000,84.456000,0.0,0.0


In [71]:
df_total5=df_total5.rename(index=str, columns={"PRICE_R": "PRICE"})

#tips = df_total5.sort_values(['Promo_ID', 'Retailer_Store_ID', 'Product_Unit_UPC'])
df_total5.head()

,P_UNITS,PRICE,TOTAL_R,SIZE,PRICE_SIZE,const,Promo_ID,Retailer_Store_ID,Product_Unit_UPC,TPP_Type,...,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R,GROUP_R,month_R,randNumCol,UNITS_R,SIZE_16,SIZE_16_PRICE
0,55.473526,3.554292,-0.048,15.0,56.1204,1.0,Brandega-24816,12345-94249-107,55444-60215,F,...,0.037,-0.027,0.025,-0.003,-0.1,0.02,1,51.408,0,0.0
1,58.530838,3.180156,-0.048,15.0,56.1204,1.0,Brandega-24816,12345-94249-107,55444-60215,F,...,0.037,-0.027,0.025,-0.003,-0.1,0.02,1,51.408,0,0.0
2,53.944870,3.741360,-0.048,15.0,56.1204,1.0,Brandega-24816,12345-94249-107,55444-60215,F,...,0.037,-0.027,0.025,-0.003,-0.1,0.02,1,51.408,0,0.0
3,57.002182,3.367224,-0.048,15.0,56.1204,1.0,Brandega-24816,12345-94249-107,55444-60215,F,...,0.037,-0.027,0.025,-0.003,-0.1,0.02,1,51.408,0,0.0
4,60.059494,2.993088,-0.048,15.0,56.1204,1.0,Brandega-24816,12345-94249-107,55444-60215,F,...,0.037,-0.027,0.025,-0.003,-0.1,0.02,1,51.408,0,0.0


In [72]:
#keep the relevant variables
df_total5 = df_total5[['Promo_ID',
 'Promo_Name',
 'PRICE_POINT',
'Retailer_Name',
 'Retailer_Store_ID',
 'Product_Group_ID',
 'Product_Unit_UPC',
 'Product_Brand',
 'Product_Brand_Category_Desc',
 'Product_Unit_Description',
 'PRICE',
 'TPP_Type',
 'TPP_Start',
 'TPP_End',
 'Product_Units_Per_Case',
 'Product_CPG_COGS_Price',
 'SIZE',
 'CASES',
 'GROUP',
 'UNITS',     
 'P_UNITS']]

In [73]:
#capitalize the field names to load in dash
df_total5=df_total5.rename(str.upper, axis='columns')

In [74]:
df_total5.PRICE_POINT.value_counts(dropna = False).sort_index()

V    134720
W    134720
X    134720
Y    134720
Z    134720
Name: PRICE_POINT, dtype: int64

In [75]:
#De-Dup the file
df_total5=df_total5.drop_duplicates(subset=['PROMO_ID','RETAILER_STORE_ID','PRODUCT_UNIT_UPC','GROUP','PRICE_POINT'], keep='first', inplace=False)
df_total5.shape
df_total5.head()


,PROMO_ID,PROMO_NAME,PRICE_POINT,RETAILER_NAME,RETAILER_STORE_ID,PRODUCT_GROUP_ID,PRODUCT_UNIT_UPC,PRODUCT_BRAND,PRODUCT_BRAND_CATEGORY_DESC,PRODUCT_UNIT_DESCRIPTION,...,TPP_TYPE,TPP_START,TPP_END,PRODUCT_UNITS_PER_CASE,PRODUCT_CPG_COGS_PRICE,SIZE,CASES,GROUP,UNITS,P_UNITS
0,Brandega-24816,Helpful Starters,V,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,...,F,2019-06-05,2019-06-13,18.0,3.06,15.0,3.0,ONSALE,54.0,55.473526
1,Brandega-24816,Helpful Starters,W,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,...,F,2019-06-05,2019-06-13,18.0,3.06,15.0,3.0,ONSALE,54.0,58.530838
2,Brandega-24816,Helpful Starters,X,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,...,F,2019-06-05,2019-06-13,18.0,3.06,15.0,3.0,ONSALE,54.0,53.944870
3,Brandega-24816,Helpful Starters,Y,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,...,F,2019-06-05,2019-06-13,18.0,3.06,15.0,3.0,ONSALE,54.0,57.002182
4,Brandega-24816,Helpful Starters,Z,Botann's,12345-94249-107,BB-BC-C-CC-15,55444-60215,Breakfast Best,Breakfast Best Cereal Cold Cereal 15 oz,Breakfast Best Plain 15 oz,...,F,2019-06-05,2019-06-13,18.0,3.06,15.0,3.0,ONSALE,54.0,60.059494


In [76]:
# The code was removed by Watson Studio for sharing.

In [77]:
df_load=df_total5

In [78]:
#load records to dashdb

import ibmdbpy
from ibmdbpy import IdaDataBase
idadb = idadb = IdaDataBase(dsn="DASHDB;Database=BLUDB;Hostname=" + credentials_1["host"] + ";PROTOCOL=TCPIP;UID=" + credentials_1["username"] + ";PWD=" + credentials_1["password"])

In [79]:
 idadb.as_idadataframe(df_load, "CEREAL_FINAL_PRODUCTION",clear_existing = True)

DataFrame will be splitted into 1773 chunks. (380 rows per chunk)
Uploaded: 1773/1773... [DONE]


In [80]:
#spark_df = sqlContext.createDataFrame(df_total5)

In [81]:
#### this inserts records


#from ingest.Connectors import Connectors

#dashdbsaveoption = {
 #                    Connectors.DASHDB.HOST              : 'dashdb-entry-yp-dal09-08.services.dal.bluemix.net',
  #                   Connectors.DASHDB.DATABASE          : 'BLUDB',
   #                  Connectors.DASHDB.USERNAME          : 'dash10112',
    #                 Connectors.DASHDB.PASSWORD          : '-rMyl8IIE1~n',
     #                Connectors.DASHDB.TARGET_TABLE_NAME : 'ICE_CREAM',
      #               Connectors.DASHDB.TARGET_WRITE_MODE : 'merge' }

#spark_df = spark_df.write.format("com.ibm.spark.discover").options(**dashdbsaveoption).save()

In [82]:
#### this deletes then inserts records


#from ingest.Connectors import Connectors

#dashdbsaveoption = {
 #                    Connectors.DASHDB.HOST              : 'dashdb-entry-yp-dal09-08.services.dal.bluemix.net',
  #                   Connectors.DASHDB.DATABASE          : 'BLUDB',
   #                  Connectors.DASHDB.USERNAME          : 'dash10112',
    #                 Connectors.DASHDB.PASSWORD          : '-rMyl8IIE1~n',
     #                Connectors.DASHDB.TARGET_TABLE_NAME : 'DASH10112.ICE_CREAM',
      #               Connectors.DASHDB.TARGET_TABLE_ACTION : 'replace' }

#spark_df = spark_df.write.format("com.ibm.spark.discover").options(**dashdbsaveoption).save()

In [83]:
#This reads records


#from ingest.Connectors import Connectors
#from pyspark.sql import SQLContext
#sqlContext = SQLContext(sc)

#dashDBloadOptions = { Connectors.DASHDB.HOST              : 'dashdb-entry-yp-dal09-08.services.dal.bluemix.net',
 #                     Connectors.DASHDB.DATABASE          : 'BLUDB',
  #                    Connectors.DASHDB.USERNAME          : 'dash10112',
   #                   Connectors.DASHDB.PASSWORD          : '-rMyl8IIE1~n',
    #                  Connectors.DASHDB.SOURCE_TABLE_NAME : 'ICE_CREAM'}

#dashdbDF = sqlContext.read.format("com.ibm.spark.discover").options(**dashDBloadOptions).load()
#dashdbDF.printSchema()
#dashdbDF.show()


In [84]:
#abcd=dashdbDF.toPandas() 

In [85]:
#abcd.shape
#abcd.info()